#This notebook implements a text mining sentiment analysis project 

##First we fetch the data from google drive

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline
import nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chris\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Read data

In [4]:
columns = ['polarity', 'tweetid', 'date', 'query_name', 'user', 'text']
dftrain = pd.read_csv(r'C:\Users\Chris\Desktop\ADS509\Project\trainingandtestdata\training.1600000.processed.noemoticon.csv',
                      header = None,
                      encoding ='ISO-8859-1')
dftest = pd.read_csv(r'C:\Users\Chris\Desktop\ADS509\Project\trainingandtestdata\testdata.manual.2009.06.14.csv',
                     header = None,
                     encoding ='ISO-8859-1')
dftrain.columns = columns
dftest.columns = columns



In [ ]:
dftrain.sample(10)

,polarity,tweetid,date,query_name,user,text
1164229,4,1979740820,Sun May 31 04:18:56 PDT 2009,NO_QUERY,bizbooks,@itaypaz i bit the bullet and signed up for #a...
1495568,4,2069955839,Sun Jun 07 16:50:22 PDT 2009,NO_QUERY,DailyCrumb,@Meonart twitter how is meon goin? and other ...
935831,4,1792844559,Thu May 14 00:26:31 PDT 2009,NO_QUERY,lexiblaire,@lsawyer17 you have officially made me a twitt...
1222579,4,1990304016,Mon Jun 01 04:29:44 PDT 2009,NO_QUERY,PicStop,@rajdeepuk you are welcome - if you need any m...
287620,0,1994286082,Mon Jun 01 11:42:05 PDT 2009,NO_QUERY,NMPrincess,"Rite now, I'm @ da mall. Gettn my hair done! I..."
704462,0,2256045842,Sat Jun 20 12:10:45 PDT 2009,NO_QUERY,AlloryB,So I have no idea why I'm such a loser can so...
1325950,4,2015196432,Wed Jun 03 03:59:30 PDT 2009,NO_QUERY,cytez,@mileycyrus i already do this
1214273,4,1989429256,Mon Jun 01 01:30:25 PDT 2009,NO_QUERY,Wrongg,"@talmklein nope, it's not your first play here..."
366754,0,2048946530,Fri Jun 05 16:02:16 PDT 2009,NO_QUERY,MikeBrooke,"Just fed my gecko, need to take my dog for a w..."
958789,4,1825826938,Sun May 17 06:57:29 PDT 2009,NO_QUERY,Vickyvargas,Prom was amazinggg!! Now off to Lake Conroe


### Text pre-processing

In [5]:
user_pat = '(?<=^|(?<=[^a-zA-Z0-9-_\.]))@([A-Za-z]+[A-Za-z0-9]+)'
http_pat = '(https?:\/\/(?:www\.|(?!www))[^\s\.]+\.[^\s]{2,}|www\.[^\s]+\.[^\s]{2,})'
repeat_pat, repeat_repl = "(.)\\1\\1+",'\\1\\1'
    

def transform(X): 
    pp_text = X
    pp_text = pp_text.str.replace(pat = user_pat, repl = 'USERNAME')
    pp_text = pp_text.str.replace(pat = http_pat, repl = 'URL')
    pp_text.str.replace(pat = repeat_pat, repl = repeat_repl)
    return pp_text

#Descriptive statistics 

In [6]:
import string
from string import punctuation

def descriptive_stats(tokens, top_num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity (https://en.wikipedia.org/wiki/Lexical_diversity), 
        and num_tokens most common tokens. Return a list with the number of tokens, number
        of unique tokens, lexical diversity, and number of characters. 
    
    """

    # Fill in the correct values here. 
    num_tokens = len(tokens)
    num_unique_tokens = len(set(tokens))
    lexical_diversity = num_unique_tokens/num_tokens
    num_characters = len("".join(tokens))

    if verbose :        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
    
        # print the five most common tokens
        
        # use a list comprehension on a set to exclude repeating
        # tokens and empty strings
        unique_tokens = [token for token in set(tokens) if token]
        # use a unique tokens to check frequency of tokens in 
        # original list
        counts = [tokens.count(token) for token in unique_tokens]
        result = []
        
        # iterate over the range of tokens to locate and find the
        # maximum count, then mutate both unique_tokens and counts
        # based on the associated position
        for _ in range(top_num_tokens):
            max_count = max(counts)
            max_count_pos = counts.index(max_count)
            most_common = unique_tokens.pop(max_count_pos)
            result.append(most_common)
            counts.pop(max_count_pos)

        print(result)

            
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters])
    


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


#PRE-Process step
#feel free to add more items to the pre-processing step
dftrain['text'] = transform(dftrain['text'])
##word_tokenize 
dftrain['text'] = dftrain.apply(lambda row: nltk.word_tokenize(row['text']), axis=1)



print("DESCRIPTIVE STATS ON TEXT: ")
all = []
#on 100k data
for li in dftrain['text'].sample(10000).iteritems(): all += li[1]
#on all data  
# for li in dftrain['text'].iteritems(): all += li[1]

#feel free to add more items to analyze in descriptive_stats
descriptive_stats(all, verbose=True)
print("\n")


print("DESCRIPTIVE STATS ON SENTIMENT POLARITY:")
dftrain['polarity'].describe()


### TODO: Train and test model

In [ ]:
Xtest, ytest = dftest.text[dftest.polarity!=2], dftest.polarity[dftest.polarity!=2]

